In [1]:
# ref: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307, ),(0.3081, ))])

train_dataset = torchvision.datasets.MNIST('./mnist_data/',
                             download=True,
                             train=True,
                             transform=transform)

test_dataset = torchvision.datasets.MNIST("./mnist_data/",
                             download=False,
                             train=False,
                             transform= transform)



trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                          shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(test_dataset, batch_size=128,
                                         shuffle=False, num_workers=2)



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net = Net().to(device)



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

cuda:0


In [2]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0
    total = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        # for train acc
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    ### train loss/acc ###
    print('[%d] loss: %.3f, train_acc: %.3f %%' %
          (epoch + 1, running_loss / 60000, 100.*correct/total))

    ### test acc ###
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('[%d] test_acc: %.3f %%' %
          (epoch + 1, 100.*correct/total))

print('Finished Training')

[1] loss: 0.002, train_acc: 90.752 %
[1] test_acc: 95.030 %
[2] loss: 0.001, train_acc: 96.205 %
[2] test_acc: 96.610 %
[3] loss: 0.001, train_acc: 97.240 %
[3] test_acc: 96.840 %
[4] loss: 0.001, train_acc: 97.823 %
[4] test_acc: 97.390 %
[5] loss: 0.000, train_acc: 98.362 %
[5] test_acc: 97.180 %
Finished Training
